***GENERATED CODE FOR filesourcejobssalaryclassification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Job Title', 'transformation_label': 'String Indexer'}], 'feature': 'Job Title', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '1.9"', 'max': 'Web Data Analyst', 'missing': '0', 'distinct': '145'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Job Title'}, {'feature_label': 'Job Title', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Job Title')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Salary Estimate', 'transformation_label': 'String Indexer'}], 'feature': 'Salary Estimate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$100K-$160K (Glassdoor est.)', 'max': 'Worcester, MA', 'missing': '0', 'distinct': '328'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Salary Estimate'}, {'feature_label': 'Salary Estimate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Salary Estimate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Rating', 'transformation_label': 'String Indexer'}], 'feature': 'Rating', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '3.77', 'stddev': '0.63', 'min': '-1', 'max': 'Zurich, Switzerland', 'missing': '0', 'distinct': '151'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Rating'}, {'feature_label': 'Rating', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Rating')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Company Name', 'transformation_label': 'String Indexer'}], 'feature': 'Company Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 to 50 employees', 'max': 'webfx.com', 'missing': '0', 'distinct': '199'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Company Name'}, {'feature_label': 'Company Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Company Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location', 'transformation_label': 'String Indexer'}], 'feature': 'Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '1894.73', 'stddev': '392.01', 'min': '-1', 'max': 'San Francisco, CA', 'missing': '250', 'distinct': '79'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location'}, {'feature_label': 'Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Headquarters', 'transformation_label': 'String Indexer'}], 'feature': 'Headquarters', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '', 'stddev': '', 'min': 'College / University', 'max': 'Unknown', 'missing': '250', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Headquarters'}, {'feature_label': 'Headquarters', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Headquarters')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Size', 'transformation_label': 'String Indexer'}], 'feature': 'Size', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '-1.0', 'stddev': 'nan', 'min': '-1', 'max': 'Wholesale', 'missing': '250', 'distinct': '51'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Size'}, {'feature_label': 'Size', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Size')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Founded', 'transformation_label': 'String Indexer'}], 'feature': 'Founded', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '1008.0', 'stddev': '1426.94', 'min': '-1', 'max': 'Travel & Tourism', 'missing': '250', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Founded'}, {'feature_label': 'Founded', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Founded')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Type of ownership', 'transformation_label': 'String Indexer'}], 'feature': 'Type of ownership', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '', 'stddev': '', 'min': '$1 to $2 billion (USD)', 'max': 'Unknown / Non-Applicable', 'missing': '250', 'distinct': '14'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Type of ownership'}, {'feature_label': 'Type of ownership', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Type of ownership')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Industry', 'transformation_label': 'String Indexer'}], 'feature': 'Industry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '0.02', 'stddev': '0.15', 'min': '0', 'max': 'IT Services', 'missing': '250', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Industry'}, {'feature_label': 'Industry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Industry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sector', 'transformation_label': 'String Indexer'}], 'feature': 'Sector', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '0.01', 'stddev': '0.09', 'min': '0', 'max': 'Information Technology', 'missing': '250', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sector'}, {'feature_label': 'Sector', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sector')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Revenue', 'transformation_label': 'String Indexer'}], 'feature': 'Revenue', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '74.33', 'stddev': '29.82', 'min': '$5 to $10 million (USD)', 'max': '99', 'missing': '250', 'distinct': '96'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Revenue'}, {'feature_label': 'Revenue', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Revenue')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'min_salary', 'transformation_label': 'String Indexer'}], 'feature': 'min_salary', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '200.0', 'stddev': 'nan', 'min': ' AL', 'max': '200', 'missing': '250', 'distinct': '36'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'min_salary'}, {'feature_label': 'min_salary', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('min_salary')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'job_state', 'transformation_label': 'String Indexer'}], 'feature': 'job_state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '0.59', 'stddev': '0.49', 'min': ' CA', 'max': '1', 'missing': '250', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'job_state'}, {'feature_label': 'job_state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('job_state')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run filesourcejobssalaryclassificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	combinedglassdoorsalaryjobs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Combined_Glassdoor_Salary_Jobs.csv', 'filename': 'Combined_Glassdoor_Salary_Jobs.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run filesourcejobssalaryclassificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(combinedglassdoorsalaryjobs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Job Title", "transformation_label": "String Indexer"}], "feature": "Job Title", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1.9\"", "max": "Web Data Analyst", "missing": "0", "distinct": "145"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Job Title"}, {"transformationsData": [{"feature_label": "Salary Estimate", "transformation_label": "String Indexer"}], "feature": "Salary Estimate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$100K-$160K (Glassdoor est.)", "max": "Worcester, MA", "missing": "0", "distinct": "328"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Salary Estimate"}, {"transformationsData": [{"feature_label": "Rating", "transformation_label": "String Indexer"}], "feature": "Rating", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "3.77", "stddev": "0.63", "min": "-1", "max": "Zurich, Switzerland", "missing": "0", "distinct": "151"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Rating"}, {"transformationsData": [{"feature_label": "Company Name", "transformation_label": "String Indexer"}], "feature": "Company Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 to 50 employees", "max": "webfx.com", "missing": "0", "distinct": "199"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Company Name"}, {"transformationsData": [{"feature_label": "Location", "transformation_label": "String Indexer"}], "feature": "Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "1894.73", "stddev": "392.01", "min": "-1", "max": "San Francisco, CA", "missing": "250", "distinct": "79"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location"}, {"transformationsData": [{"feature_label": "Headquarters", "transformation_label": "String Indexer"}], "feature": "Headquarters", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "", "stddev": "", "min": "College / University", "max": "Unknown", "missing": "250", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Headquarters"}, {"transformationsData": [{"feature_label": "Size", "transformation_label": "String Indexer"}], "feature": "Size", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "-1.0", "stddev": "nan", "min": "-1", "max": "Wholesale", "missing": "250", "distinct": "51"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Size"}, {"transformationsData": [{"feature_label": "Founded", "transformation_label": "String Indexer"}], "feature": "Founded", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "1008.0", "stddev": "1426.94", "min": "-1", "max": "Travel & Tourism", "missing": "250", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Founded"}, {"transformationsData": [{"feature_label": "Type of ownership", "transformation_label": "String Indexer"}], "feature": "Type of ownership", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "", "stddev": "", "min": "$1 to $2 billion (USD)", "max": "Unknown / Non-Applicable", "missing": "250", "distinct": "14"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Type of ownership"}, {"transformationsData": [{"feature_label": "Industry", "transformation_label": "String Indexer"}], "feature": "Industry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "0.02", "stddev": "0.15", "min": "0", "max": "IT Services", "missing": "250", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Industry"}, {"transformationsData": [{"feature_label": "Sector", "transformation_label": "String Indexer"}], "feature": "Sector", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "Information Technology", "missing": "250", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sector"}, {"transformationsData": [{"feature_label": "Revenue", "transformation_label": "String Indexer"}], "feature": "Revenue", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "74.33", "stddev": "29.82", "min": "$5 to $10 million (USD)", "max": "99", "missing": "250", "distinct": "96"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Revenue"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hourly", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "250", "mean": "126.42", "stddev": "44.61", "min": "0", "max": "275", "missing": "250"}, "updatedLabel": "hourly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "employer_provided", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "250", "mean": "100.23", "stddev": "36.75", "min": "1.0", "max": "237.5", "missing": "250"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "employer_provided"}, {"transformationsData": [{"feature_label": "min_salary", "transformation_label": "String Indexer"}], "feature": "min_salary", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "200.0", "stddev": "nan", "min": " AL", "max": "200", "missing": "250", "distinct": "36"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "min_salary"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "max_salary", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "250", "mean": "1.57", "stddev": "15.78", "min": "0", "max": "250", "missing": "250"}, "updatedLabel": "max_salary"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "avg_salary", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "250", "mean": "44.79", "stddev": "53.87", "min": "-1.0", "max": "239.0", "missing": "250"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "avg_salary"}, {"transformationsData": [{"feature_label": "job_state", "transformation_label": "String Indexer"}], "feature": "job_state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "0.59", "stddev": "0.49", "min": " CA", "max": "1", "missing": "250", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "job_state"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "same_state", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "250", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "250"}, "updatedLabel": "same_state"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "250", "mean": "0.26", "stddev": "0.47", "min": "0", "max": "3", "missing": "250"}, "updatedLabel": "age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "python_yn", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "250", "mean": "0.22", "stddev": "0.42", "min": "0", "max": "1", "missing": "250"}, "updatedLabel": "python_yn"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "R_yn", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "250", "mean": "0.56", "stddev": "0.5", "min": "0", "max": "1", "missing": "250"}, "updatedLabel": "R_yn"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "spark", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "0.0", "stddev": "nan", "min": "0", "max": "0", "missing": "499"}, "updatedLabel": "spark"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "aws", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "1.0", "stddev": "nan", "min": "1", "max": "1", "missing": "499"}, "updatedLabel": "aws"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "excel", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1", "mean": "1.0", "stddev": "nan", "min": "1", "max": "1", "missing": "499"}, "updatedLabel": "excel"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run filesourcejobssalaryclassificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["hourly", "employer_provided", "max_salary", "avg_salary", "same_state", "age", "python_yn", "R_yn", "spark", "aws", "excel", "Job Title_stringindexer", "Rating_stringindexer", "Company Name_stringindexer", "Location_stringindexer", "Headquarters_stringindexer", "Size_stringindexer", "Founded_stringindexer", "Type of ownership_stringindexer", "Industry_stringindexer", "Sector_stringindexer", "Revenue_stringindexer", "min_salary_stringindexer", "job_state_stringindexer"], "Salary Estimate_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

